In [1]:
# this is to analyze proteins of Human, Brain and NDD phenotypes creating arrays of intersection and unition of these.
import numpy as np
import pandas as pd
import config as cfg
import brain as brn
import dataframe_image as dfi
import seaborn as sns

def phens_acc_dict_maker(phen_lst, phens_df):  # TODO: later try to use these two methods inside eachother
    phen_dict = dict.fromkeys(phen_lst)  # this dict has phen_name as key and lst of corresponding ACCs as values
    for each_phen in phen_lst:
        tmp_lst = list(set(phens_df.loc[phens_df['phenotype'] == each_phen, 'acc'].tolist()))
        phen_dict[each_phen] = tmp_lst
    return phen_dict


def human_brain_acc_adder(phens_dict):
    ## Mobidb downloaded on Aug 8th 2022, 78,106 entries (but 77629 prs after filtering for disorder consensus)
    mobidb = pd.read_csv(cfg.data['clin'] + '/mobidb_result_2022-08-08T13_12_39.379Z.tsv', sep='\t', usecols=['acc', 'feature'])
    mobidb = mobidb.loc[mobidb['feature'] == 'prediction-disorder-th_50']
    human_lst = list(set(mobidb['acc'].tolist()))
    brain_lst = brn.brain_pr_lst_generator()
    brain_lst = list(set(brain_lst) & set(human_lst))
    phens_dict['Human'] = human_lst
    phens_dict['Brain'] = brain_lst
    return phens_dict


def phens_intersect_df_maker(phen_acc_dic):
    # gets dictionary with phen as key and their uniprot IDs as values,
    intersection_df = pd.DataFrame(index=phen_acc_dic.keys(), columns=phen_acc_dic.keys())
    for key1 in phens_acc_dict.keys():
        intersection_count_lst = []
        for key2 in phens_acc_dict.keys():
            count_tmp = len(list(set(phens_acc_dict[key1]) & set(phens_acc_dict[key2])))
            intersection_count_lst.append(count_tmp)
        intersection_df[key1] = intersection_count_lst
    return intersection_df


def phens_union_df_maker(phen_acc_dic):
    union_df = pd.DataFrame(index=phen_acc_dic.keys(), columns=phen_acc_dic.keys())
    for key1 in phens_acc_dict.keys():
        union_count_lst = []
        for key2 in phens_acc_dict.keys():
            count_tmp = len(list(set(phens_acc_dict[key1]) | set(phens_acc_dict[key2])))
            union_count_lst.append(count_tmp)
        union_df[key1] = union_count_lst
    return union_df

In [3]:
phens_lst = ['Human', 'Brain', 'ASD', 'Ep', 'ID', 'ADHD', 'Cancer', 'SCZ', 'T2D']
phens_subdf = pd.read_csv(cfg.data['hc'] + '/smaller-hc-with-phens-column')
# Dictionary with phen as key and their corresponding  list of ACCs as value
phens_acc_dict = human_brain_acc_adder(phens_acc_dict_maker(phens_lst, phens_subdf))
phens_inters_df = phens_intersect_df_maker(phens_acc_dict)  
phens_union_df = phens_union_df_maker(phens_acc_dict)
## in_idr_vars similarity 
cm = sns.light_palette("green", as_cmap=True)

dfi.export(phens_inters_df, 'new-intersection.png')
table_similarity = (phens_inters_df *100 / phens_union_df).style.background_gradient(cmap=cm).set_precision(2)
dfi.export(table_similarity, 'new-similarity.png')


/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:23: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  brain_df = pd.read_table(cfg.data['brain'] + '/proteinatlas.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:28: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  normal_tissue_df = pd.read_table(cfg.data['brain'] + '/normal_tissue.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  brain_filtered_df = brain_filtered_df.append(df_temp)  # (161558, 6)
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will b

In [4]:
## old
phens_lst = ['Human', 'Brain', 'ASD', 'ID', 'Ep', 'Cancer']
clin_mobidb_ndd = pd.read_csv(cfg.data['clin'] + '/vars-in_and_out_idr-checked-by-mobidb.csv')
in_idr_vars = clin_mobidb_ndd.loc[clin_mobidb_ndd['isin_idr'] == 1]
# Dictionary with phen as key and their corresponding  list of ACCs as value
phens_acc_dict = human_brain_acc_adder(phens_acc_dict_maker(phens_lst, in_idr_vars))
phens_inters_df = phens_intersect_df_maker(phens_acc_dict)  
phens_union_df = phens_union_df_maker(phens_acc_dict)
## in_idr_vars similarity 
cm = sns.light_palette("green", as_cmap=True)
# Overlap (Jaccard index)
table_in_idr = (phens_inters_df *100 / phens_union_df).style.background_gradient(cmap=cm).set_precision(2)
dfi.export(table_in_idr, 'similarity-IDR-vars.png')

/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:23: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  brain_df = pd.read_table(cfg.data['brain'] + '/proteinatlas.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:28: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  normal_tissue_df = pd.read_table(cfg.data['brain'] + '/normal_tissue.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  brain_filtered_df = brain_filtered_df.append(df_temp)  # (161558, 6)
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will b

In [5]:
## old
phens_lst = ['Human', 'Brain', 'ASD', 'ID', 'Ep', 'Cancer']
clin_mobidb_ndd = pd.read_csv(cfg.data['clin'] + '/vars-in_and_out_idr-checked-by-mobidb.csv')
out_idr_vars = clin_mobidb_ndd.loc[clin_mobidb_ndd['isin_idr'] == 0]
# Dictionary with phen as key and their corresponding  list of ACCs as value
phens_acc_dict = human_brain_acc_adder(phens_acc_dict_maker(phens_lst, out_idr_vars))
phens_inters_df = phens_intersect_df_maker(phens_acc_dict)  
phens_union_df = phens_union_df_maker(phens_acc_dict)
## out_idr_vars similarity 
cm = sns.light_palette("green", as_cmap=True)
# Overlap (Jaccard index)
table_out_idr = (phens_inters_df *100 / phens_union_df).style.background_gradient(cmap=cm).set_precision(2)
dfi.export(table_out_idr, 'similarity-OR-vars.png')

/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:23: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  brain_df = pd.read_table(cfg.data['brain'] + '/proteinatlas.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:28: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  normal_tissue_df = pd.read_table(cfg.data['brain'] + '/normal_tissue.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  brain_filtered_df = brain_filtered_df.append(df_temp)  # (161558, 6)
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will b

In [9]:
## old
phens_lst = ['Human', 'Brain', 'ASD', 'Ep', 'ID', 'ADHD', 'Cancer']
clin_mobidb_ndd = pd.read_csv(cfg.data['clin'] + '/vars-in_and_out_idr-checked-by-mobidb.csv')
# Dictionary with phen as key and their corresponding  list of ACCs as value
phens_acc_dict = human_brain_acc_adder(phens_acc_dict_maker(phens_lst, clin_mobidb_ndd))
phens_inters_df = phens_intersect_df_maker(phens_acc_dict)  
phens_union_df = phens_union_df_maker(phens_acc_dict)
## all_vars similarity 
cm = sns.light_palette("green", as_cmap=True)
# Overlap (Jaccard index)
all_vars_table = (phens_inters_df *100 / phens_union_df).style.background_gradient(cmap=cm).set_precision(2)
dfi.export(all_vars_table, 'similarity-all-vars.png')

/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:23: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  brain_df = pd.read_table(cfg.data['brain'] + '/proteinatlas.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:28: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  normal_tissue_df = pd.read_table(cfg.data['brain'] + '/normal_tissue.tsv', error_bad_lines=False, sep='\t')
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  brain_filtered_df = brain_filtered_df.append(df_temp)  # (161558, 6)
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/brain.py:37: FutureWarning: The frame.append method is deprecated and will b

In [10]:
#TODO do comparison between in idr and out idr based on var IDs 

In [11]:

intersect_t = phens_intersect_df_maker(phens_acc_dict).style.background_gradient(cmap=cm).set_precision(2)
dfi.export(intersect_t, 'intersect-count-all.png')

/tmp/ipykernel_6117/2732752527.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  intersect_t = phens_intersect_df_maker(phens_acc_dict).style.background_gradient(cmap=cm).set_precision(2)
/home/fatemeh/PycharmProjects/IDPs_and_NDDs/src/venv/lib/python3.10/site-packages/dataframe_image/_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'
[0926/101706.959848:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0926/101707.011785:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0926/101707.294902:INFO:headless_shell.cc(660)] Written to file /tmp/tmp02k8ztrp/temp.png.
